In [ ]:
# Importing necessary tensorflow libraries
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint

from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Assuming you have defined tensorboard_callback somewhere in your code
tensorboard_callback = ...

data_dir = Path('Dataset/Train')
filepaths = list(data_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=42)

In [ ]:

batch_size = 4
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    labels='inferred',
    label_mode='int',
    y_col='Label',
    target_size=(256, 256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=123,
    cros_to_aspect_ratio=False,
    follow_links=False,
    interpolation='bilinear',
    subset='training'
)

val_generator = test_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    label_mode='int',
    labels='inferred',
    y_col='Label',
    target_size=(256, 256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    interpolation='bilinear',
    shuffle=True,
    seed=123,
    follow_links=False,
    cros_to_aspect_ratio=False,
    subset='validation'
)

# no. of classes
num_classes = 14
# Using InceptionV3 as the base model
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3),
)
# Creating our layers on top of base model
inputs = Input(shape=(256, 256, 3))
x = base_model(inputs)
x = GlobalAveragePooling2D()(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs, outputs)

# Adam Optimizer used for training
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)
# Compiling the model
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(),
    metrics=[
        CategoricalAccuracy(),
        TopKCategoricalAccuracy(k=3)
    ]
)

# Creating checkpoints
model_checkpoint = ModelCheckpoint(
    filepath="model_l_train_checkpoints",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch"
)

# No. of epochs
epochs = 100
# Fitting the model to the data
model.fit(
    x=train_generator,
    epochs=epochs,
    steps_per_epoch=72520 // 32,
    verbose=2,
    callbacks=[tensorboard_callback, model_checkpoint],
    validation_data=val_generator,
    validation_steps=1475 // 16,
)

# Evaluate on train and validation generators
train_evaluation = model.evaluate(train_generator, steps=len(train_generator))
val_evaluation = model.evaluate(val_generator, steps=len(val_generator))

print("Train Evaluation:", train_evaluation)
print("Validation Evaluation:", val_evaluation)